# Secciones Eficaces y Branching Ratios

## Secciones Eficaces

In [1]:
Sigma_ttbar_13TeV={'value':835,'stat':3,'syst':23,'lumi':23,'source':"http://cms-results.web.cern.ch/cms-results/public-results/preliminary-results/TOP-16-022/index.html"};
Sigma_ttbar_13TeV={'value':835,'stat':3,'syst':23,'lumi':23,'source':"http://cms-results.web.cern.ch/cms-results/public-results/preliminary-results/TOP-16-022/index.html"};

#Sigma_={'value':,'stat':,'syst':,'lumi':};

## Branching Ratios

In [2]:
Br_Wlnu={'value':10.83,'stat':0.07,'syst':0.07,'source':"Porcentual PDG Chinese_Phys_C_40_1000001 Pag 617"};
Br_Wmunu={'value':10.63,'stat':0.15,'syst':0.12,'source':'Porcentual PDG Chinese_Phys_C_40_1000001 Pag 617'}
Br_tWb={'value':0.91,'stat':0.04,'syst':'NA','source':'Porcentual PDG Chinese_Phys_C_40_1000001 Pag 617'}

### Función para generar las tablas

In [23]:
def TABLA( Z, formato= '.3e'):
    """Formato es el formato de los numeros, ejem: .2f , .4e"""
    tabla=[];
    head=Z.keys();
    head.sort();
    tmp=Z[Z.keys()[0]].keys();
    tmp.sort();
    for i in tmp:
        entries=[];
        entries.append(i);
        for j in head:
            entries.append(Z[j][i]);
        tabla.append(entries);
    head.insert(0,'Cuts');
    tabla.insert(0,head);
    
    return HTML(tabulate(tabla, headers= "firstrow", floatfmt= formato, tablefmt='html'))

# Análisis de datos utilizando PyRoot


###Cargo los paquetes nesesarias

In [4]:
#from IPython.core.interactiveshell import InteractiveShell
import ROOT
import os
import commands
import math
####################TABLES########################################################
from tabulate import tabulate
from IPython.display import display, Math, Latex
from IPython.display import HTML
#InteractiveShell.ast_node_interactivity = "all" # Para tener el modo interactivo completo

###Cargo las Librerías nesesarias

In [5]:
ROOT.gROOT.Reset()
#Esta es la ruta a la librería dinámica de Delphes (debe de ser compilada con la misma versión de root del notebook)
#Solo funciona con la versión de Delphes compilada con root 5 (y obiamente root 5)
ROOT.gSystem.AddDynamicPath("~/HEPTools/Delphes/delphes5/")
ROOT.gSystem.Load("libDelphes");

###Construcción de los TChain

In [6]:
#Lista de carpetas le analisis
Path1= "/scratch/cms/Datos/" #Inportante dejar aquí el último /
Path2= "/Events"
Path3= "/tag_1_delphes_events.root"
ListOfFolders = ["BackGround-DY_2j-Run0","BackGround-tW-Run0","BackGround-WW-Run0","Signal_2j_mu-nu-Run0","BackGround-mumuW"]


In [7]:
#Para guardar la info sobre los cortes
signal={};
for Folder in ListOfFolders:
    comand ="ls " + Path1 + Folder + Path2;
    tmp = commands.getstatusoutput( comand )
    
    #Parto la cadena de caracteres por '\n', y asigno las partes a un arreglo
    Runs = tmp[1].split("\n");
    
    #Creo el objeto TChain
    MainChain=ROOT.TChain("Delphes")
    
    #Lleno el objeto MainChain con las salidas de MadGraph-Pythia_Delphes
    for run in range(len(Runs)):
        imput = Path1 + Folder + Path2 + "/" + Runs[run] + "/tag_1_delphes_events.root"
        MainChain.Add(imput)
        
    #Creo el objeto ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(MainChain)
    numberOfEntries = treeReader.GetEntries()
    
    #Obtengo las ramas que voy a usar
    branchJet = treeReader.UseBranch("Jet")
    branchMuon = treeReader.UseBranch("Muon")
    branchMissingET = treeReader.UseBranch("MissingET")

    # Inicializo los contadores
    cut1=0; cut2=0; cut3=0; cut4=0; cut5=0;
    cuts={};
    
    # Loop sobre los eventos
    for entry in range(0, numberOfEntries):
        #Se carga la entrada espesífica
        treeReader.ReadEntry(entry)
        
        # Primer corte, al menos 2 jet
        if branchJet.GetEntries() > 1:
            cut1 = cut1 +1;
            bcount=0;
            for i in range(0, branchJet.GetEntries() ): #Esta parte es la que produce el error (TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available)
                jet = branchJet.At(i)
                #Cuento cuantos jet hay identificados como b quarks
                if jet.BTag :
                    bcount = bcount + 1;
            
            # Segundo corte, al menos 2 jet b       
            if bcount > 1:
                cut2 = cut2 +1;
            
                # If event contains at least 2 muons
                if branchMuon.GetEntries() > 1:
                    cut3 = cut3 +1;
                    # Tomo los dos muones mas energéticos y comparo sus cargas eléctricas
                    muon1 = branchMuon.At(0)
                    muon2 = branchMuon.At(1)
                
                    #Corte 4: Que los muones mas energéticos tengan cargas diferentes
                    if muon1.Charge*muon2.Charge < 0:
                        cut4 = cut4 +1;

                    
                        #Corte 5: muones mas energéticos tengan masa invariante diferente de la del Z
                        MassMuon= (muon1.P4()+muon2.P4()).M();
                        if not((MassMuon>80) and (MassMuon<100)):
                            cut5 = cut5 +1;
    

    cuts['C0']=numberOfEntries;
    cuts['C1'] = cut1;
    cuts['C2'] = cut2;
    cuts['C3'] = cut3;
    cuts['C4'] = cut4;
    cuts['C5'] = cut5; 
    signal[Folder]=cuts

TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available


Donde Co,C1,C2,... Son el número de eventos, sin cortes, con el primer corte, segundo y asi susecivamente.

In [8]:
EffA={};  #Eficiencia Acumulada
for entry in signal.keys():
    EffCut={};  
    for i in signal[entry].keys():
        EffCut[i] = (0.1*signal[entry][i])/(0.1*signal[entry]['C0']);
    EffA['EffA_'+entry] = EffCut;

In [9]:
Effr={};  #Eficiencia Relativa
a=signal[entry].keys()
a.sort()
for entry in signal.keys():
    EffCut={};
    j=0;
    a1=a[j];
    for i in a[1:]:
        EffCut[i] = (0.1*signal[entry][i])/(0.1*signal[entry][a1]);
        a1=a[j];
        j=j+1;
    Effr['Effr_'+entry] = EffCut;

In [10]:
Z={};  #Sensitividad
a=signal.keys()
a.remove('Signal_2j_mu-nu-Run0')
a.sort()
B=0;
for entry in a:
    Zi={};  
    b=signal[entry].keys()
    b.sort()
    for i in signal[entry].keys():
        Zi[i] = (0.1*signal['Signal_2j_mu-nu-Run0'][i])/(math.sqrt((0.1*signal['Signal_2j_mu-nu-Run0'][i])+(0.1*signal[entry][i])));
        B=(0.1*signal[entry][i])+B;
    Z["Sensitivity_"+entry] = Zi;
    ZZ = (0.1*signal['Signal_2j_mu-nu-Run0'][i])/(math.sqrt((0.1*signal['Signal_2j_mu-nu-Run0'][i])+B));

In [20]:
TABLA(signal)

Cuts,BackGround-DY_2j-Run0,BackGround-WW-Run0,BackGround-mumuW,BackGround-tW-Run0,Signal_2j_mu-nu-Run0
C0,160448,85553,325964,80417,355148
C1,28247,74134,238907,76727,341827
C2,317,25633,511,27754,126190
C3,137,11387,196,12751,55688
C4,137,11387,196,12750,55687
C5,14,9838,10,10985,48077


In [26]:
TABLA(Effr, '.3f')

Cuts,Effr_BackGround-DY_2j-Run0,Effr_BackGround-WW-Run0,Effr_BackGround-mumuW,Effr_BackGround-tW-Run0,Effr_Signal_2j_mu-nu-Run0
C1,0.176,0.867,0.733,0.954,0.962
C2,0.002,0.300,0.002,0.345,0.355
C3,0.005,0.154,0.001,0.166,0.163
C4,0.432,0.444,0.384,0.459,0.441
C5,0.102,0.864,0.051,0.862,0.863


In [28]:
TABLA(Effr, '.3f')

Cuts,Effr_BackGround-DY_2j-Run0,Effr_BackGround-WW-Run0,Effr_BackGround-mumuW,Effr_BackGround-tW-Run0,Effr_Signal_2j_mu-nu-Run0
C1,0.176,0.867,0.733,0.954,0.962
C2,0.002,0.300,0.002,0.345,0.355
C3,0.005,0.154,0.001,0.166,0.163
C4,0.432,0.444,0.384,0.459,0.441
C5,0.102,0.864,0.051,0.862,0.863


In [29]:
TABLA(Z, '.3f')

Cuts,Sensitivity_BackGround-DY_2j-Run0,Sensitivity_BackGround-WW-Run0,Sensitivity_BackGround-mumuW,Sensitivity_BackGround-tW-Run0
C0,156.406,169.175,136.082,170.170
C1,177.690,167.602,141.846,167.082
C2,112.193,102.413,112.108,101.705
C3,74.533,67.996,74.493,67.315
C4,74.532,67.995,74.493,67.314
C5,69.327,63.175,69.330,62.558
